In [3]:
import pandas  as pd

In [4]:
df = pd.read_csv(
    r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\raw_dataset\2025\20250301_Production\20250301_Equities_feed_new_strategies_filtered_old_names_iso_permId.csv",
    usecols=[
            "isin",
            "instrument_type",
            "issuer_name",
            "issuer_country",
            "permId"]
             
    dtype={
            "isin": str,
            "permId": str
        }
    )   

In [6]:
df.rename(columns={"permId": "permid"}, inplace=True)

In [5]:
#get crossreference
crossreference = pd.read_csv(r"C:\Users\n740789\Documents\Projects_local\DataSets\crossreference\Aladdin_Clarity_Issuers_20250301.csv", 
                             dtype={"CLARITY_AI": str})

crossreference.columns = ["brs_id", "brs_issuer_name", "permid", "msci_id", "sustainalytics_id"]

In [7]:
# add "brs_id" to df from crossreference
df = pd.merge(df, crossreference[["brs_id", "permid"]], on="permid", how="left")

ValueError: You are trying to merge on int64 and object columns for key 'permid'. If you wish to proceed you should use pd.concat

In [4]:
for col in df.columns[:10]:
    print(col)

isin
instrument_type
issuer_name
issuer_country
gics2_industry
region
company_inheriting
parent_company
esg_score
esg_score_relevance
